In [1]:
import math
import pandas as pd
import numpy as np
import pandas_datareader as web
import matplotlib.pyplot as plt
import math
from datetime import date
from datetime import datetime

In [6]:
# Get The price data for all of our stocks
start_date = '2010-01-04'
end_date = '2010-12-18'
## Whirlpool Stock. Self Explanatory
whr_stock = web.DataReader('WHR', data_source = 'yahoo', start = start_date, end = end_date)
# Ferro Corporation is a supplier of coatings for whirlpool
foe_stock = web.DataReader('FOE', data_source = 'yahoo', start = start_date, end = end_date)
# Worthington Industries is a steel supplier for whirlpool
wor_stock = web.DataReader('WOR', data_source = 'yahoo', start = start_date, end = end_date)
# Exxon Mobile Chemical Corporation supplies chemicals to whirlpool
xom_stock = web.DataReader('XOM', data_source = 'yahoo', start = start_date, end = end_date)
# Lowes is where whirlpool sells their products
low_stock = web.DataReader('LOW', data_source = 'yahoo', start = start_date, end = end_date)
#HD is also a place where products are sold
hd_stock = web.DataReader('HD', data_source = 'yahoo', start = start_date, end = end_date)
#BASFY is a chemical supplier
basfy_stock = web.DataReader('BASFY', data_source = 'yahoo', start = start_date, end = end_date)
mhk_stock = web.DataReader('MHK', data_source = 'yahoo', start = start_date, end = end_date)

# These two are people who ship for Whirlpool
pag_stock = web.DataReader('PAG', data_source = 'yahoo', start = start_date, end = end_date)
jbht_stock = web.DataReader('JBHT', data_source = 'yahoo', start = start_date, end = end_date)

In [7]:
# Find the start and end of continous time periods in data
### We treat holidays as weekends, so not all weeks are the same time length for now
weeks = []
index = 0
labels = list(whr_stock.index)
while index < (whr_stock.shape[0]) - 1:
    start_date = index
    not_found = True
    end_date = start_date
    while not_found:
        old_index = index
        if index != (whr_stock.shape[0] - 1):
            index = index + 1
            time_dif = (labels[index] - labels[old_index]).days
            if time_dif > 1:
                not_found = False
                end_date = old_index
        else: 
            end_date = index
            not_found = False
    weeks.append([start_date, end_date])

In [8]:
#Now we need to process the stock prices we have. We are looking at how price changes from 1 period affect
# Whirlpool's stock in the next period
## The first step is to define a function to help create 3 states of each stock: fall, steady, and raise
# Lets first define a function to consolidate our scores. 
# if the % change in a week is between +-1%, we assign it a 1. If it is <-1%, we assign a 0. If it is >1%, we assign it 2
def standardize(percent):
    if (percent > -.01) and (percent < .01):
        return 1
    elif percent > 0:
        return 2
    else:
        return 0

In [66]:
# Now we calculate percentage change over time periods
## Now we create our list of price changes
### We will create the WHR one in a seperate vector from the rest since we
### are going to "shift" the whirlpool one since we are trying to predict future price
data_points_percent = []
data_points_state = []
time_labels = []
whr_future_percent = []
whr_future_state = []
for time in weeks:
    start_val_whr = whr_stock.iloc[time[0]]
    end_val_whr = whr_stock.iloc[time[1]]
    difference_whr = (start_val_whr.get('Open') - end_val_whr.get('Close') ) / start_val_whr.get('Open')
    #MHK
    start_val_mhk = mhk_stock.iloc[time[0]]
    end_val_mhk = mhk_stock.iloc[time[1]]
    difference_mhk = (start_val_mhk.get('Open') - end_val_mhk.get('Close') ) / start_val_mhk.get('Open')
    #PAG
    start_val_pag = pag_stock.iloc[time[0]]
    end_val_pag = pag_stock.iloc[time[1]]
    difference_pag = (start_val_pag.get('Open') - end_val_pag.get('Close') ) / start_val_pag.get('Open')
    #HD
    start_val_hd = hd_stock.iloc[time[0]]
    end_val_hd = hd_stock.iloc[time[1]]
    difference_hd = (start_val_hd.get('Open') - end_val_hd.get('Close') ) / start_val_hd.get('Open')
    #JBHT
    start_val_jbht = jbht_stock.iloc[time[0]]
    end_val_jbht = jbht_stock.iloc[time[1]]
    difference_jbht = (start_val_jbht.get('Open') - end_val_jbht.get('Close') )/ start_val_jbht.get('Open')
    #LOW
    start_val_low = low_stock.iloc[time[0]]
    end_val_low = low_stock.iloc[time[1]]
    difference_low = (start_val_low.get('Open') - end_val_low.get('Close'))  / start_val_low.get('Open')
    #FOE
    start_val_foe = foe_stock.iloc[time[0]]
    end_val_foe = foe_stock.iloc[time[1]]
    difference_foe = (start_val_foe.get('Open') - end_val_foe.get('Close')) / start_val_foe.get('Open')
    #WOR
    start_val_wor = wor_stock.iloc[time[0]]
    end_val_wor = wor_stock.iloc[time[1]]
    difference_wor = (start_val_wor.get('Open') - end_val_wor.get('Close')) / start_val_wor.get('Open')
    #XOM
    start_val_xom = xom_stock.iloc[time[0]]
    end_val_xom = xom_stock.iloc[time[1]]
    difference_xom = (start_val_xom.get('Open') - end_val_xom.get('Close')) / start_val_xom.get('Open')
    #BASFY
    start_val_basfy = basfy_stock.iloc[time[0]]
    end_val_basfy = basfy_stock.iloc[time[1]]
    difference_basfy = (start_val_basfy.get('Open') - end_val_basfy.get('Close')) / start_val_basfy.get('Open')
    #combine everything
    percent_point = [difference_whr, difference_mhk, difference_pag,
                     difference_hd, difference_jbht, difference_low,
                     difference_foe, difference_wor, difference_xom,
                    difference_basfy]
    state_point = list(map(standardize, percent_point))
    data_points_percent.append(percent_point)
    data_points_state.append(state_point)
    whr_future_percent.append(difference_whr)
    whr_future_state.append(standardize(difference_whr))
    #Time Labels
    time_labels.append(end_val_whr.name)\
## We adjust the data a bit so that the future value of whirlpool is offset backwards by 1 week
data_points_percent.pop()
data_points_state.pop()
time_labels.pop()
whr_future_percent.pop(0)
whr_future_state.pop(0)

0

In [70]:
# Add turn into data frame and add labels
column_labels = ['WHR-C', 'MHK', 'PAG', 'HD', 'JBHT', 'LOW', 'FOE', 'WOR', 'XOM', 'BASFY', 'WHR-F']

data_percent = pd.DataFrame(data_points_percent, index = time_labels)
data_state = pd.DataFrame(data_points_state, index = time_labels)
temp = pd.Series(whr_future_percent)
data_percent = data_percent.assign(e = temp.values)
temp = pd.Series(whr_future_state)
data_state = data_state.assign(e = temp.values)
data_percent.columns = column_labels
data_state.columns = column_labels
print(data_percent.head())
print(data_state.head())

               WHR-C       MHK       PAG        HD      JBHT       LOW  \
2010-01-08 -0.039076 -0.028382 -0.098554  0.005832 -0.045455 -0.003403   
2010-01-15  0.029710  0.062513  0.028708  0.014828  0.000588  0.019915   
2010-01-22  0.054024  0.037631  0.079454  0.026001  0.054561  0.032943   
2010-01-29  0.052433  0.094666  0.062041 -0.002147  0.052534  0.036064   
2010-02-05 -0.006479 -0.064192 -0.043909  0.001071  0.011028  0.005985   

                 FOE       WOR       XOM     BASFY     WHR-F  
2010-01-08 -0.068966 -0.299625 -0.011641  0.007039  0.029710  
2010-01-15  0.042286  0.076180  0.011867  0.058888  0.054024  
2010-01-22  0.052821  0.071385  0.038406  0.061667  0.052433  
2010-01-29  0.020202  0.051148  0.031856  0.004386 -0.006479  
2010-02-05  0.074264  0.008230  0.014748  0.052926 -0.065363  
            WHR-C  MHK  PAG  HD  JBHT  LOW  FOE  WOR  XOM  BASFY  WHR-F
2010-01-08      0    0    0   1     0    1    0    0    0      1      2
2010-01-15      2    2    2   2  

In [71]:
data_percent.to_csv('percent_changes.csv', encoding='utf-8', index=True)
data_state.to_csv('state_changes.csv', encoding='utf-8', index=True)
